In [ ]:
%matplotlib inline
import re
import scipy
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Разреженные признаки

**Примеры** источников разреженных признаков:

* Категориальные признаки 
* Текст

## Категориальные признаки

Категориальные признаки, также могут упоминаться, как факторные или номинальные признаки. 

**Примеры** категориальных признаков: 

* пол
* страна проживания
* номер группы

и т.п.

Ясно, что для компьютерной обработки вместо "понятного для человека" значения (в случае страны — "Russia", "GB", "France" и т.п.) хранят числа. Далее обсудим, как получать подобное вектор-признаки и в каком формате их хранить.

Рассмотрим таблицу ```winemag.csv```, которая содержит описания вин.

In [ ]:
data = pd.read_csv('winemag.csv', index_col=0, na_filter=False)
data.head()

В таблице 10 столбцов с признаками. Какие из них являются категориальными? 

Во-первых, это должны быть столбцы содержащие текстовые значения. Следовательно, в качестве кандидатов остаются: ```country```, ```description```, ```designation```, ```province```, ```region_1```, ```region_2```, ```variety``` и ```winery```.

Во-вторых, столбцы с небольшим числом уникальных значений:

In [ ]:
for name in ['country', 'description', 'designation', 'province',
             'region_1', 'region_2', 'variety', 'winery']:
    print('%s: %d'%(name, data[name].nunique()))

In [ ]:
np.unique(data["points"])

Итак, страна-производитель является категориальным признаком.

Самым очевидным способом кодирования является - **one-hot-кодирование**: для кодируемого категориального признака создаются $N$ новых признаков, где $N$ - число категорий. Каждый $i$-й новый признак - бинарный характеристический признак $i$-й категории.

Например, страна-производитель является категориальным признаком. Воспользуемся [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) и [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) для преобразования названий стран в one-hot-вектор.

In [ ]:
countries = data.country
countries = LabelEncoder().fit_transform(countries)
countries = OneHotEncoder().fit_transform(countries[:, np.newaxis])

type(countries)

## Разреженные матрицы

После кодирования мы получили разреженную матрицу признаков. Существует много типов разреженных матриц, каждый из которых предоставляет разные гарантии на операции.

* ```scipy.sparse.coo_matrix```
* ```scipy.sparse.csc_matrix```
* ```scipy.sparse.csr_matrix```
* ```scipy.sparse.bsr_matrix```
* ```scipy.sparse.lil_matrix```
* ```scipy.sparse.dia_matrix```
* ```scipy.sparse.dok_matrix```

Подробнее про [устройство разреженых матрицы](http://www.netlib.org/utk/people/JackDongarra/etemplates/node372.html)

### scipy.sparse.coo_matrix

* Используется как хранилище данных
* Поддерживает быструю конвертацию в любой формат
* Не поддерживает индексацию
* Поддерживает ограниченый набор арифметических операций

### scipy.sparse.csc_matrix

* Хранит данные поколоночно
* Быстрое получение значений отдельных колонок

### scipy.sparse.csr_matrix

* Хранит данные построчно
* Быстрое получение значений отдельных строк

### scipy.sparse.bsr_matrix

* Подходит для разреженных матриц с плотными подматрицами

### scipy.sparse.lil_matrix

* Подходит для создания разреженных матриц поэлементно
* Для последующих матричных операций лучше сконвертировать в ```csr_matrix``` или ```csc_matrix```

Библиотека ```scipy.sparse``` содержит методы, позволяющие работать с разреженными матрицами. Подробнее про операции с разрежеными матрицами на сайте [scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html).

## Предсказание оценки вин

В качестве категориальных признаков возьмём: country, province и variety

Попробуем предсказать оценку выставленную винам. Оценки в таблицы варьируются от 80 до 100.

In [ ]:
Y = data.points.values
countries = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.country)[:, np.newaxis])
provinces = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.province)[:, np.newaxis])
varieties = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.variety)[:, np.newaxis])
features = [('country', countries), ('province', provinces), ('variety', varieties)]

names = []
accuracy_scores = []
for subset_features in tqdm(itertools.chain(*[list(itertools.combinations(features, n)) 
                                              for n in range(1, 4)])):
    subset_names, subset_features = zip(*subset_features)
    names.append('; '.join(subset_names))
    
    X = scipy.sparse.hstack(subset_features)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    
    lr = LogisticRegression().fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)

    accuracy_scores.append(accuracy_score(Y_pred, Y_test))

pd.DataFrame({'Accuracy':accuracy_scores}, index=names)

## Извлечение признаков из текстов

Перед тем как работать с текстом, его необходимо токенизировать - разбить на отдельные токены. В качестве токенов могут выступать слова, фразы, предложений и т.п. Токенизировать текст можно  помощью регулярных выражений или готовых токенизаторов. 

После токенизации нужно привести текст к нормальной форме. Речь идет о [стемминге и/или лемматизации](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) - это схожие процессы, используемые для обработки словоформ.

Для работы лемматизации английского текста можно воспользоваться библиотекой [SpaCy]( https://spacy.io/):

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en')
description_lemma = [' '.join([token.lemma_ for token in nlp(text)]) 
                     for text in tqdm(data.description)]

In [ ]:
description_lemma[0]

### Bag of Words

Cоздаем вектор длиной в словарь, для каждого слова считаем количество вхождений в текст и подставляем это число на соответствующую позицию в векторе.

Построим модель BOW с помощью [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
vectorizer = CountVectorizer().fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Видно, что большая часть из топ-10 слов является не информативными - стоп-словами. Что бы они не участвовали в представление, в конструктор CountVectorizer в качестве параметра можно передать список стоп-слов:

In [ ]:
from stop_words import get_stop_words

In [ ]:
stop_words = get_stop_words('en')
vectorizer = CountVectorizer(stop_words=stop_words).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Чтобы сжать векторное представление, можно "отбросить" редкие слова:

In [ ]:
vectorizer = CountVectorizer(stop_words=stop_words, min_df=3).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
description_count

### Tf-Idf

Cлова, которые редко встречаются в корпусе (во всех рассматриваемых документах этого набора данных), но присутствуют в этом конкретном документе, могут оказаться более важными. Тогда имеет смысл повысить вес более узкотематическим словам, чтобы отделить их от общетематических. Этот подход называется [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf).

Значение Tf-Idf для каждого пары документ-слово состоит из двух компонент:
* Term frequency — логарифм встречаемости слова в документе

$$tf(t, d) = \log n_{t,d}$$

* Inverse Document frequency — логарифм обратной доли документов в которых встретилось данное слово

$$idf(t, D) = \log \frac{ \mid D \mid}{\mid \{ d_i \in D \mid t \in d_i \} \mid}$$

* Tf-Idf — кобминация tf и idf

$$ TfIdf(t, d, D) = tf(t, d) * idf(t, D)$$

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()

description_tfidf = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

## Предсказание оценки вин

Добавляем к категориальным признакам, признаки извлечённые из описаний сомелье.

In [ ]:
accuracy_scores = []
for description in [description_count, description_tfidf]:

    X = scipy.sparse.hstack([countries, provinces, varieties, description])
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    
    lr = LogisticRegression().fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)
    
    accuracy_scores.append(accuracy_score(Y_pred, Y_test))

pd.DataFrame({'Accuracy':accuracy_scores}, index=['BOW', 'TfIdf'])

### BM25

Метод вычисления весов ```Okapi BM25```, который развивает идею Tf-Idf, учитывая длину документов в $tf(t, d)$.

$$tf(t, d) = \frac{(k_1 + 1) * n_{t,d}}{k_1 * (1 - b + b * \frac{L_d}{L_{ave}}) + n_{t,d}}$$

где
* $k_1$ и $b$ - свободные коэффициенты, обычно их выбирают как $k_1=2.0$ и $b=0.75$
* $L_d$ - длина документа $d$
* $L_{ave}$ - средняя длина документа в коллекции